## Import Statements

In [91]:
import re
import json
import csv
import boto3
import time
import pandas as pd
result = {}
import os

#### Make Sure that the "saved_folder" is already created in the same S3 bucket

### INPUT: enter initial document details

In [92]:
#initial document details
s3Name = #s3BucketName
docuName = #document name
saved_folder = #Folder to put textract outputs
box_file_name = #Box Name
csv_file_name = #Name of CSV output file

## Textract

#### Assuming API (Boto3) is already authenticated
#### refer to https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html for Boto3 authenticatoin
#### refer to https://docs.aws.amazon.com/textract/latest/dg/async.html for textract documentation

In [93]:
startJob("fei-rfc-bucket-training1", "Somu_Cards_34.pdf")

'daaca3f1ff6eb61e0aee77c9f443bd63614e52de0724ea739c9d4a47f804aae8'

In [94]:
#start Textract job
def startJob(s3BucketName, objectName):
    response = None
    client = boto3.client(service_name='textract', region_name='us-west-2')
    response = client.start_document_text_detection(
    DocumentLocation={
        'S3Object': {
            'Bucket': s3BucketName,
            'Name': objectName
        }
    })

    return response["JobId"]

In [95]:
#get current status of the Textract job, checking in 5 second intervals
def isJobComplete(jobId):
    time.sleep(5)
    client = boto3.client(service_name='textract', region_name='us-west-2')
    response = client.get_document_text_detection(JobId=jobId)
    status = response["JobStatus"]
    print("Job status: {}".format(status))

    while(status == "IN_PROGRESS"):
        time.sleep(5)
        response = client.get_document_text_detection(JobId=jobId)
        status = response["JobStatus"]
        print("Job status: {}".format(status))

    return status

In [96]:
#once job is complete get the results
def getJobResults(jobId):
    pages = []
    time.sleep(5)
    client = boto3.client(service_name='textract', region_name='us-west-2')
    response = client.get_document_text_detection(JobId=jobId)
    pages.append(response)
    print("Resultset page recieved: {}".format(len(pages)))
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']

    while(nextToken):
        time.sleep(5)
        response = client.get_document_text_detection(JobId=jobId, NextToken=nextToken)
        pages.append(response)
        print("Resultset page recieved: {}".format(len(pages)))
        nextToken = None
        if('NextToken' in response):
            nextToken = response['NextToken']

    return pages

In [97]:
#start textract and check if job is complete
def start_textract(s3BucketName, documentName):
    jobId = startJob(s3BucketName, documentName)
    print("Started job with id: {}".format(jobId))
    if(isJobComplete(jobId)):
        response = getJobResults(jobId)
        print ('uploading...')

    return(response)

In [98]:
#upload the finished textract files back to the s3 bucket
#this is needed for comprehend to run later
def upload_text1(response):
    text=[]
    page=[]
    file_name = "Output1.txt"
    text_file = open(file_name, "w")
    count = 0
    for resultPage in response:
        for item in resultPage["Blocks"]:
            if item["BlockType"] == "LINE":
                text.append(item["Text"])
                page.append(item['Page'])
                if page[count] == page[count-1]:
                    text_file.write(item["Text"] + '\n')
                    count=count+1
                else:
                    text_file.close()
                    s3_client = boto3.client('s3')
                    response = s3_client.upload_file(file_name, s3Name, saved_folder + "/{}".format(file_name))
                    file_name = "Output" + str(item['Page']) + ".txt"
                    text_file = open(file_name, "w")
                    text_file.write(item["Text"] +'\n')
                    count=count+1

    #print(text)
    text_file.close()
    print('finished')

In [101]:
#main function that calls other helper functions
def convert_card_image():
    result = start_textract(s3Name, docuName)
    upload_text1(result)

### RUN: function below to start textract

In [102]:
#run this call to start textract and upload the files to s3 bucket when finished
convert_card_image()

Started job with id: 6db308f8614fd949099771dc8ff6245bd764a43c5303311b830baf8d5f274268
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
uploading...
[{'DocumentMetadata': {'Pages': 34}, 'JobStatus': 'SUCCEEDED', 'NextToken': 'zeXYs8FTp1N9wEMZfH/mxkke5w0Ql+arAYANFDLRyHa2yF8ez0tl7KFSbsbPJrzEqRg+vQp+S+qsGy+UqHsLyUREru0UALousvtvEQQaG0kNCppN9g==', 'Blocks': [{'BlockType': 'PAGE', 'Geometry': {'BoundingBox': {'Width': 0.9640164971351624, 'Height': 0.9074740409851074, 'Left': 0.016541900113224983, 'Top': 0.0345025435090065}, 'Polygon': [{'X': 0.016541900113224983, 'Y': 0.04175826907157898}, {'X': 0.9805583953857422, 'Y': 0.0345025435090065}, {'X': 0.9803820252418518, 'Y': 0.9385411143302917}, {'X': 0.01697682775557041, 'Y': 0.9419766068458557}]}, 'Id': 'f456c844-57e2-4e6b-a183-464d034e9526', 'Relationships': [{'Type': 'CHILD', 'Ids': ['69e472a3-9d84-44f7-b7df-27c1724ddca2', '69b877b2-3466-4a7c-99d9-ab11fcef6072', 'e4884

### INPUT: AWS Comprehend
#### The below values are sample inputs give. The dARN, input, and outputs are specific to your model and file locations. 
#### For more information, refer: https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectEntities.html#comprehend-DetectEntities-request-EndpointArn

In [70]:
#general comprehend
general_dARN = "arn:aws:iam::448546609819:role/service-role/AmazonComprehendServiceRole-Box6Model"
general_s3_input = "s3://fei-rfc-bucket-training1/Cards_OutputFiles_Testy/"
general_s3_output = "s3://fei-rfc-bucket-training1/RFC-Test/ComprehendOutput3/"

#decision comprehend
decision_dARN = "arn:aws:iam::448546609819:role/service-role/AmazonComprehendServiceRole-Box6Model"
decision_ARN = "arn:aws:comprehend:us-west-2:448546609819:entity-recognizer/DecisionRFC"
decision_s3_input = "s3://fei-rfc-bucket-training1/Cards_OutputFiles_Testy/"
decision_s3_output = "s3://fei-rfc-bucket-training1/RFC-Test/ComprehendOutput3/"

#custom comprehend
custom_dARN = "arn:aws:iam::448546609819:role/service-role/AmazonComprehendServiceRole-Box6Model"
custom_ARN = "arn:aws:comprehend:us-west-2:448546609819:entity-recognizer/BankLocationNew"
custom_s3_input = "s3://fei-rfc-bucket-training1/Cards_OutputFiles_Testy/"
custom_s3_output = "s3://fei-rfc-bucket-training1/RFC-Test/ComprehendOutput3/"

### Start Comprehend Models

#### refer to https://docs.aws.amazon.com/comprehend/latest/dg/functionality.html for general comprehend documentation
#### refer to https://docs.aws.amazon.com/comprehend/latest/dg/auto-ml.html for custom comprehend documentation

In [71]:
#starts general comprehend analysis job
def start_general_comprehend():
    comprehend = boto3.client(service_name='comprehend', region_name='us-west-2')
    response = comprehend.start_entities_detection_job(
        DataAccessRoleArn = general_dARN,
        LanguageCode = "en",
        InputDataConfig= { 
            "InputFormat": "ONE_DOC_PER_LINE",
            "S3Uri": general_s3_input  
        },
        OutputDataConfig= { 
            "S3Uri": general_s3_output
        }
    )
    return response["JobId"]

In [72]:
#starts custom comprehend analysis job (decision entitiy only) 
def start_decision_comprehend():
    comprehend = boto3.client(service_name='comprehend', region_name='us-west-2')
    response = comprehend.start_entities_detection_job(
        DataAccessRoleArn = decision_dARN,
        EntityRecognizerArn= decision_ARN,
        LanguageCode = "en",
        InputDataConfig= { 
            "InputFormat": "ONE_DOC_PER_LINE",
            "S3Uri": decision_s3_input 
        },
        OutputDataConfig= { 
            "S3Uri": decision_s3_output
        }
    )
    return response["JobId"]

In [73]:
#starts custom comprehend analysis job (other entities) 
def start_custom_comprehend():
    comprehend = boto3.client(service_name='comprehend', region_name='us-west-2')
    response = comprehend.start_entities_detection_job(
        DataAccessRoleArn = custom_dARN,
        EntityRecognizerArn= custom_ARN,
        LanguageCode = "en",
        InputDataConfig= { 
            "InputFormat": "ONE_DOC_PER_LINE",
            "S3Uri": custom_s3_input
        },
        OutputDataConfig= { 
            "S3Uri": custom_s3_output
        }
    )
    return response["JobId"]

In [74]:
#get current status of the comprehend job, checking in 20 second intervals
def isJobComplete_comprehend(jobID, type_comprehend):
    time.sleep(1)
    client = boto3.client(service_name='comprehend', region_name='us-west-2')
    response = client.describe_entities_detection_job(JobId=jobID)
    
    while(response['EntitiesDetectionJobProperties']['JobStatus'] == "IN_PROGRESS"):
        response = client.describe_entities_detection_job(JobId=jobID)
        print("working...")
        time.sleep(20)
    
    return response['EntitiesDetectionJobProperties']['JobStatus']

In [75]:
#function to start all comprehends together and print when each is done
def start_all_comprehend():
    general_id = start_general_comprehend()
    decision_id = start_decision_comprehend()
    comprehend_id = start_custom_comprehend()
    general = isJobComplete_comprehend(general_id, "general")
    decision = isJobComplete_comprehend(decision_id, "decision")
    comprehend = isJobComplete_comprehend(comprehend_id, "comprehend")
    print(general)
    print(decision)
    print(comprehend)
    
    

### RUN: Start all comprehend functions

In [76]:
#calling function from above
start_all_comprehend()

working...
working...
working...
working...
working...
working...
working...
working...
working...
working...
working...
working...
working...
working...
working...
working...
working...
working...
working...
working...
working...
working...
working...
working...
working...
working...
COMPLETED
COMPLETED
COMPLETED


### INPUT: Comprehend Output File Names

In [77]:
custom_text_output = "Custom.txt"
general_text_output = "General.txt"
decision_text_output = "Decision.txt"

### Parse through Comprehend Outputs

In [78]:
#parse through general comprehend and identify banks or locations
def initial_comprehend(file_name):
    final_output = {}
    file1 = open("General.txt")
    Lines = file1.readlines()
    for line in Lines:
        line_dict = json.loads(line)
        if line_dict["File"] == file_name:
            final_output[line_dict["Line"]] = {}
            for entitiy in line_dict["Entities"]:
                final_output[line_dict["Line"]][entitiy["Type"]] = entitiy["Text"]
    return final_output

In [79]:
def if_normal(file_name):
    custom_results = ["No", "No"]
    basic = initial_comprehend(file_name)
    for line in basic:
        if custom_results[0] == "No":
            if "ORGANIZATION" in basic[line]:
                custom_results[0] = basic[line]["ORGANIZATION"]
        if custom_results[1] == "No":
            if "LOCATION" in basic[line]:
                custom_results[1] = basic[line]["LOCATION"]
    return custom_results

In [80]:
#parse through custom comprehend and identify decision
def if_decision (file_name, line_num):
    file1 = open("Decision.txt")
    Lines = file1.readlines()
    bank = "NA"
    decision = ""
    for line in Lines:
        line_dict = json.loads(line)
        if line_dict["File"] == file_name:
            if line_dict["Line"] == line_num:
                for text_type in line_dict["Entities"]:
                    if "Type" in text_type:
                        if text_type["Type"] == "Decision":
                            decision = text_type["Text"]
    return decision

In [81]:
#parse through custom comprehend
def if_custom(file_name):
    file1 = open("Custom.txt")
    Lines = file1.readlines()
    bank = "No"
    loc = "No"
    for line in Lines:
        line = str(line)
        line_dict = json.loads(line)
        if line_dict["File"] == file_name:
            for text_type in line_dict["Entities"]:
                if "Type" in text_type:
                    if text_type["Type"] == "BANK":
                        bank = text_type["Text"]
                    elif text_type["Type"] == "LOCATION":
                        loc = text_type["Text"]
    return ([bank, loc])

In [82]:
#compare custom and general comprhend outputs to see if they are similar
def compare(file_name):
    custom_results = if_custom(file_name)
    normal_results = if_normal(file_name)
    if custom_results[0] == "No":
        custom_results[0] = normal_results[0]
    if custom_results[1] == "No":
        custom_results[1] = normal_results[1]
    return custom_results

### Regex Functions

In [83]:
#returns abbrev if one exists within the line number (line_num) of the file (file_name)
def abbrev_finder(file_name, line_num):
    file1 = open(file_name, 'r')
    Lines = file1.readlines()
    
    line = Lines[line_num]
    abbrev = re.search('[A-z]+.?-.?[0-9]+',line)
    if abbrev:
        return([abbrev.group(0), line])
    else:
        return [None,line]

In [84]:
#returns paid if one exists within the line number (line_num) of the file (file_name)
def paid_finder(file_name, num):
    file1 = open(file_name, 'r')
    Lines = file1.readlines()
    line = Lines[num]
    paid = re.search('PAID+',line)
    if paid:
        return paid.group(0)
    else:
        return ""

In [85]:
#returns bank if one exists within the line number (line_num) of the file (file_name)
def get_bank(file_name):
    file1 = open(file_name, 'r')
    Lines = file1.readlines()
    for line in Lines:
        bank2 = re.search('^Bank.*', line)
        if bank2:
            if "Loan" not in bank2.group(0):
                return(bank2.group(0).split(" - ")[0])
        else:
            bank2 = re.search('^bank.*',line)
            if bank2:
                if "Loan" not in bank2.group(0):
                    return(bank2.group(0).split(" - ")[0])
        bank = re.search('.+?(?=Bank)',line)
        if bank:
            return(bank.group(0) + "Bank")
        else:
            bank = re.search('.+?(?=bank)',line)
            if bank:
                return(bank.group(0) + "Bank")
            else:
                continue
    return("")

In [86]:
#identifies if county is in the line
def county_in(file_name):
    file1 = open(file_name)
    Lines = file1.readlines()
    for line in Lines:
        if 'County' in line or 'county' in line:
            return [1,line]
            
    return [0,""]

### Combine Regex + Comprehend Functions

In [87]:
#combine regex + comprehend outputs to get all identified entities 
def rest_finder(file):
    file2 = open('output18.txt','r')
    lines = file2.readlines()
    for line in lines:
        line_dict = json.loads(line)
        line_int = int(line_dict["Line"])
        line_number = "line: " + str(line_dict["Line"])
        result[line_number] = {}
        for entity in line_dict["Entities"]:
            result[line_number][entity["Type"]] = entity["Text"]
            result[line_number]["Abbrev"] = abbrev_finder(line_int)
            result[line_number]["Paid"] = paid_finder(line_int)[0]
            result[line_number]["Line"] = paid_finder(line_int)[1]
            result[line_number]["Page Number"] = paid_finder(line_int)[2]
    #return (result)
    decision()
    return (result)

### Compile into CSV

In [88]:
#compile outputs into one singular csv file
def get_csv_test():
    wd = os.getcwd()
    files = [i for i in os.listdir(wd) if i.endswith("txt")]
    with open (csv_file_name, 'w', newline = '') as file:
        writer = csv.writer(file)
        writer.writerow(["Bank Name", "Location", "Date", "Abbrev", "Decision", "Amount", "Paid", "Box Number", "Page Number", "Line", "Bank_Confidence", "Loc_Confidence", "Line", "County", "County Text"])
        counter = 1
        for file_name in files:
            if file_name == "Decision.txt" or file_name == "BankLocCode.txt":
                continue
            print("Working on " + str(counter))
            counter += 1
            both = compare(file_name)
            custom = if_custom(file_name)
            normal = if_normal(file_name)
            if custom[0] == normal[0]:
                bank_c = 1
            else:
                bank_c = 0
            if custom[1] == normal[1]:
                loc_c = 1
            else:
                loc_c = 0
            general = initial_comprehend(file_name)
            for line in general:
                date = ""
                abbrev = ""
                decision = ""
                amount = ""
                paid = ""
                box_number = box_file_name
                page_number = re.search('[0-9]+',file_name).group(0)
                line_str = ""
                line_text = ""
                if 'DATE' in general[line] or "QUANTITY" in general[line]:
                    if 'DATE' in general[line]:
                        date = general[line]['DATE']
                    if 'QUANTITY' in general[line]:
                        amount = general[line]['QUANTITY']
                    print(file_name + " , " + str(line))
                    abbrev_total = abbrev_finder(file_name, int(line))
                    abbrev = abbrev_total[0]
                    paid = paid_finder(file_name, int(line))
                    decision = if_decision(file_name, int(line))
                    line_str = str(int(line))
                    line_text = [abbrev_total[1]][0]
                    county = county_in(file_name)
                    writer.writerow([both[0], both[1], date, abbrev, decision, amount, paid, box_number, page_number, line_str, str(bank_c), str(loc_c), line_text, county[0], county[1]])

### RUN: function to download csv

In [89]:
#calling above function
get_csv_test()

Working on 1
Output10.txt , 2
Working on 2
Output2.txt , 3
Output2.txt , 4
Output2.txt , 5
Output2.txt , 6
Output2.txt , 7
Working on 3
Output3.txt , 2
Output3.txt , 4
Output3.txt , 5
Working on 4
Output11.txt , 4
Output11.txt , 5
Output11.txt , 6
Output11.txt , 7
Working on 5
Output13.txt , 2
Working on 6
Output1.txt , 3
Working on 7
Output12.txt , 2
Working on 8
Output16.txt , 2
Working on 9
Output4.txt , 3
Output4.txt , 4
Working on 10
Output5.txt , 3
Output5.txt , 4
Output5.txt , 5
Output5.txt , 6
Output5.txt , 7
Working on 11
Output17.txt , 2
Output17.txt , 3
Output17.txt , 4
Output17.txt , 5
Output17.txt , 6
Working on 12
Output15.txt , 2
Working on 13
Output29.txt , 3
Working on 14
Working on 15
Output7.txt , 3
Working on 16
Output6.txt , 2
Working on 17
Output28.txt , 3
Working on 18
Output14.txt , 3
Output14.txt , 4
Working on 19
Working on 20
Working on 21
Output19.txt , 2
Working on 22
Output31.txt , 3
Output31.txt , 4
Working on 23
Output25.txt , 2
Working on 24
Output24.tx